# 04. Roadmap Generator

Сурагчийн түвшин, сул талуудад үндэслэн хувийн сургалтын төлөвлөгөө үүсгэх

**Input:**
- Түвшин (1-10)
- Сул сэдвүүд
- Зорилтот оноо
- Боломжит хугацаа (долоо хоног)

**Output:**
- Долоо хоног тус бүрийн төлөвлөгөө

In [ ]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import List, Dict
import joblib

In [ ]:
# Topic dependencies (урьдчилсан нөхцөл)
TOPIC_DEPENDENCIES = {
    'algebra': [],
    'geometry': ['algebra'],
    'trigonometry': ['geometry', 'algebra'],
    'calculus': ['algebra', 'trigonometry'],
    'mechanics': ['algebra', 'trigonometry'],
    'thermodynamics': ['algebra'],
    'electromagnetism': ['algebra', 'calculus'],
    'organic': ['algebra'],
    'inorganic': [],
    'physical_chemistry': ['algebra', 'thermodynamics']
}

# Topic resources
TOPIC_RESOURCES = {
    'algebra': ['Алгебрын үндэс', 'Тэгшитгэл бодох', 'Функц'],
    'geometry': ['Планиметр', 'Стереометр', 'Координатын геометр'],
    'trigonometry': ['Тригонометрийн функц', 'Тригонометрийн тэгшитгэл'],
    'calculus': ['Лимит', 'Дифференциал', 'Интеграл'],
    'mechanics': ['Кинематик', 'Динамик', 'Энерги'],
    'thermodynamics': ['Термодинамикийн хууль', 'Дулааны машин'],
    'electromagnetism': ['Цахилгаан орон', 'Соронзон орон'],
    'organic': ['Нүүрсустөрөгч', 'Спирт, хүчил'],
    'inorganic': ['Элемент', 'Нэгдэл'],
    'physical_chemistry': ['Химийн тэнцвэр', 'Кинетик']
}

In [ ]:
@dataclass
class WeekPlan:
    week_number: int
    topics: List[str]
    goals: List[str]
    resources: List[str]
    daily_practice: int  # Өдөрт бодох бодлогын тоо


class RoadmapGenerator:
    def __init__(self):
        self.dependencies = TOPIC_DEPENDENCIES
        self.resources = TOPIC_RESOURCES
    
    def _topological_sort(self, topics: List[str]) -> List[str]:
        """Сэдвүүдийг dependency-ийн дарааллаар эрэмбэлэх"""
        visited = set()
        result = []
        
        def dfs(topic):
            if topic in visited:
                return
            visited.add(topic)
            for dep in self.dependencies.get(topic, []):
                if dep in topics:
                    dfs(dep)
            result.append(topic)
        
        for topic in topics:
            dfs(topic)
        
        return result
    
    def generate(self, level: int, weak_topics: List[str], 
                 target_score: int = 700, weeks_available: int = None) -> List[WeekPlan]:
        """
        Roadmap үүсгэх
        
        Args:
            level: Одоогийн түвшин (1-10)
            weak_topics: Сул сэдвүүд
            target_score: Зорилтот оноо
            weeks_available: Боломжит долоо хоног
        """
        # Calculate weeks needed
        if weeks_available is None:
            weeks_available = max(4, (10 - level) * 2)
        
        # Sort topics by dependencies
        sorted_topics = self._topological_sort(weak_topics)
        
        # Calculate daily practice based on level
        daily_practice = 5 + (10 - level)  # Lower level = more practice
        
        plans = []
        topics_per_week = max(1, len(sorted_topics) // weeks_available + 1)
        
        for week in range(1, weeks_available + 1):
            start_idx = (week - 1) * topics_per_week
            end_idx = min(start_idx + topics_per_week, len(sorted_topics))
            
            week_topics = sorted_topics[start_idx:end_idx] if start_idx < len(sorted_topics) else ['Нийт давталт']
            
            # Goals
            if week <= len(sorted_topics) // topics_per_week:
                goals = [
                    f"{', '.join(week_topics)} сэдвийг гүнзгий судлах",
                    f"Өдөрт {daily_practice} бодлого бодох",
                    "Алдаагаа тэмдэглэж шинжлэх"
                ]
            else:
                goals = [
                    "Бүх сэдвээр давтлага хийх",
                    "Шалгалтын бодлого бодох",
                    "Цаг хугацааны менежмент"
                ]
            
            # Resources
            resources = []
            for t in week_topics:
                resources.extend(self.resources.get(t, []))
            
            plan = WeekPlan(
                week_number=week,
                topics=week_topics,
                goals=goals,
                resources=resources[:5],  # Limit resources
                daily_practice=daily_practice
            )
            plans.append(plan)
        
        return plans

In [ ]:
# Test the generator
generator = RoadmapGenerator()

roadmap = generator.generate(
    level=5,
    weak_topics=['algebra', 'trigonometry', 'mechanics'],
    target_score=700
)

for week in roadmap:
    print(f"\n=== Долоо хоног {week.week_number} ===")
    print(f"Сэдвүүд: {', '.join(week.topics)}")
    print(f"Зорилтууд: {week.goals}")
    print(f"Өдөрт бодох: {week.daily_practice} бодлого")

In [ ]:
# Save generator
joblib.dump(generator, '../trained_models/roadmap_generator.pkl')
print('Roadmap generator saved!')